# QubitiseBox Examples
* Given a LCUBox, it constructs a QSVTOracleBox by reflecting it about the |00...0> state on the prepare qubits by applying a CnZ gate on the prepare register.
* Includes smart decompositions of the controlled QubitiseBox and the controlled squared QubitiseBox.

Let us consider a toy example:

In [1]:
from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

hamiltonian = QubitPauliOperator(
        {
            QubitPauliString([Qubit(0), Qubit(1)], [Pauli.I, Pauli.I]): 0.5,
            QubitPauliString([Qubit(0), Qubit(1)], [Pauli.X, Pauli.Z]): 0.1,
        }
    )

n_state_qubits = 2

Now we construct the LCUBox and the QubitiseBox:

In [2]:
from qtnmtts.circuits.lcu import LCUMultiplexorBox
from qtnmtts.circuits.qubitisation import QubitiseBox

lcu_box = LCUMultiplexorBox(hamiltonian, n_state_qubits)
qubitise_box = QubitiseBox(lcu_box)

In [3]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(qubitise_box.get_circuit())

Now when doing the controlled operation observe that the prepare circuit does not need to be controlled:

In [4]:
render_circuit_jupyter(qubitise_box.qcontrol(1).get_circuit())

## Squared QubitiseBox
In case of requiring a power of 2 ** n of the QubitiseBox, its controlled implementation is the following one given the LCUBox is hermitian:

In [5]:
render_circuit_jupyter(qubitise_box.power(4).qcontrol(1).get_circuit())

Notice here the LCUBox does not need to be controlled. Alternatively, one can decide to control the LCU instead of the reflection. This equivalence always holds but may require more gates depending on the LCU:

In [6]:
qubitise_box_2 = QubitiseBox(lcu_box)
qubitise_box_2.control_reflection = False
render_circuit_jupyter(qubitise_box_2.power(4).qcontrol(1).get_circuit())

Let's see that both implementations are equivalent:

In [7]:
import numpy as np
np.isclose(qubitise_box.power(4).qcontrol(1).get_unitary(), qubitise_box_2.power(4).qcontrol(1).get_unitary()).all()

True

Notice that it is also possible to call first qcontrol and later power and still the implementation is the desired one:

In [8]:
render_circuit_jupyter(qubitise_box.qcontrol(1).power(4).get_circuit())

## Powers $\ne 2^n$
In case of not having powers of the shape of 2**n, one can construct the circuit with squared and single components of QubitiseBox. For instance, assume we want a power of 5:

In [9]:
qcontrol_qubitise_box = qubitise_box.qcontrol(1)

circ = qcontrol_qubitise_box.initialise_circuit()

circ.add_registerbox(qcontrol_qubitise_box.power(4))
circ.add_registerbox(qcontrol_qubitise_box.power(1))

render_circuit_jupyter(circ)

In [10]:
# Check that the operation is equivalent
np.isclose(circ.get_unitary(), qcontrol_qubitise_box.power(5).get_unitary()).all()

True